# 모듈

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Kkma
k = Kkma()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

from chatspace import ChatSpace
spacer = ChatSpace()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 워드 클라우드
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from IPython.display import set_matplotlib_formats
matplotlib.rc('font',family = 'Malgun Gothic')
set_matplotlib_formats('retina')
matplotlib.rc('axes',unicode_minus = False)

Loading JIT Compiled ChatSpace Model


In [2]:
def find_word_idx(word):
    cnt = 0
    cnt_l1 = []
    for i in rew['morphs']:
        if word in i:
            cnt_l1.append(cnt)
        cnt += 1
    return cnt_l1

def find_word_sen(word):
    cnt = 0
    cnt_l1 = []
    for i in rew['rew_pre']:
        if word in i:
            cnt_l1.append(cnt)
        cnt += 1
    return cnt_l1

def find_pos_idx(word):
    cnt = 0
    cnt_l1 = []
    for i in rew['pos']:
        if word in i:
            cnt_l1.append(cnt)
        cnt += 1
    return cnt_l1

#def find_noun_idx(word):
#    cnt = 0
#    cnt_l1 = []
#    for i in rew['rew_n']:
#        if word in i:
#            cnt_l1.append(cnt)
#        cnt += 1
#    return cnt_l1

def word_replace(word, replace_word):
    rew['rew_pre'].iloc[find_word_sen(word)] = rew['rew_pre'].iloc[find_word_sen(word)].map(lambda x: str(x).replace(word,replace_word))

# 데이터 불러오기

In [3]:
rew = pd.read_csv('../dataset/glowpick_reviews.csv')

In [4]:
glow = pd.read_csv('../dataset/glowpick_products.csv')

# 데이터 확인

In [5]:
print('shape:',rew.shape)
print('\n#### 결측치 갯수 ####\n')
print(rew.isnull().sum())
rew.head()

shape: (5208, 7)

#### 결측치 갯수 ####

date             0
user_id          0
sex              0
age_skin_type    0
rate             0
content          0
product_url      0
dtype: int64


,date,user_id,sex,age_skin_type,rate,content,product_url
0,2개월 전,현콕,m,28세 · 복합성 ·,good,제일 무난하고 가볍게쓰기 좋아서 몇년째 이거만써요,/product/7212
1,2개월 전,sso___,f,28세 · 복합성 ·,soso,남자친구 선물로 사줬는데 그저그렇데요 나쁘지도 좋지도 않데요,/product/7212
2,3개월 전,minseo6666,f,17세 · 복합성 ·,good,아빠 사드렸는데 저아해여,/product/7212
3,4개월 전,피부에관심있는남자,m,21세 · 복합성 ·,good,향이 좋아요 근데 좀 제 피부가 민감해서 안맞는거 같아요 좀 올라오더라고요ㅠ,/product/7212
4,5개월 전,타라타라,f,24세 · 복합성 ·,good,가격이 저렴하고 냄새가 안 쎄서 좋아요. 👍 다음에도 또 구매할 것 같아요.,/product/7212


# 전처리

In [6]:
np.array([i.split('·') for i in rew['age_skin_type']])[:,0]
rew['age']= np.array([i.split('·') for i in rew['age_skin_type']])[:,0]
rew['skin_type']= np.array([i.split('·') for i in rew['age_skin_type']])[:,1]

In [7]:
del rew['age_skin_type']

# 리뷰 전처리

In [8]:
print('리뷰 총 갯수: {}'.format(rew['content'].shape[0]))

리뷰 총 갯수: 5208


In [9]:
def preprocessing(text):
    # 개행문자 제거(줄바꿈 문자)
    text = re.sub('\n', ' ', text)
    # 의성어 제거
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ!?~]', ' ',text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기서는 제거
    # re.sub : 첫번째 인수에 바꿀 문자열을 입력하고 두 번째 인수에 대상 문자열을 입력한다.
    # text = re.sub('[?.,;:|\)*~'!^\- +<>@\#$%&-=#}*]','',text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]',' ',text)
    # [^5]와 같이 '^'가 []안에서 쓰이는 경우에는 5를 제외한 모든 문자를 나타냄
    # 가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z을 만족하는 것들을 제외한 것들을 ' '로 치환
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9,.]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +',' ',text)
    return text

- .의 유무에 따라(문장단위로 나눠지냐) 뒤에오는 단어 형태가 달라져서 ,.를 남겨둠

In [10]:
rew['content'] = rew['content'].map(str)

In [11]:
rew['rew_pre'] = rew['content'].apply(preprocessing)

In [12]:
rew['rew_pre'] = rew['rew_pre'].map(lambda x: x.upper())

In [13]:
rew['rew_pre'].head()

0                   제일 무난하고 가볍게쓰기 좋아서 몇년째 이거만써요
1             남자친구 선물로 사줬는데 그저그렇데요 나쁘지도 좋지도 않데요
2                                 아빠 사드렸는데 저아해여
3    향이 좋아요 근데 좀 제 피부가 민감해서 안맞는거 같아요 좀 올라오더라고요 
4      가격이 저렴하고 냄새가 안 쎄서 좋아요. 다음에도 또 구매할 것 같아요.
Name: rew_pre, dtype: object

## 띄어쓰기 모델

In [14]:
rew['rew_pre'] = rew['rew_pre'].map(lambda x: spacer.space(x))

## 단어변환

In [15]:
rew['rew_pre'][132] = '비오템 입문 하자마자 떠나게 한 제품 피부에 신경안쓰던적에 올인원 제품 하나만 겨울에 바를만큼 무신경했는데 요 라인은 참을수 없이 건조했다 '
rew['rew_pre'][2435] = '최악중최악 바르면 얼굴따갑고 아침엔트러블로가득찬 내얼굴 한번쯤 여드름쟁이가 되고 싶다면 추천 하는 크림'
rew['rew_pre'][2818]= '내가 흑우인지 괜히 면도세안 한꺼번에 되는 폼이라길래 구매했다가 제대로 돈날린 제품 쫀쫀이고 쫀득이고 간에 일단 샤워하면서는 절대 쓸수 없었다 어찌된 영문인지 물에 상당히 약해서 얼굴이 세면후 상태로 촉촉하면 죄다 녹아 내려서 폼이 오일로 바뀌는 매직을 겪게된다 샤워 전 최저한의 수분으로 거품유지하며 씻는 스트레스에 매번 조절 실패해서 제품낭비 돈낭비 제대로 한 역대급 최악 클랜징 제품 지금은 차세안 하기위해서 그냥 세안 전 한번 사용해주고 전용 클랜징폼은 따로 쓴다 오히려 면도크림 용으로만 나왔었어도 평균은 가지 않았을까 하는 느낌 상당히 화한 느낌으로 트러블 없이 면도가 가능하긴 한데 이걸 면도크림만으로 쓰기엔 가격면에서 상당히 그냥 따로 면도할렵니다 '
rew['rew_pre'][2822] = '내 인생 최고의 똥템'
rew['rew_pre'][147] = '원래 sk 를 사용하다가 갈아탔는데 순한거같기는 하나 그렇게 피부가 좋아지는데 도움을 주는거는 크게 못느꼈어요 무난한 사용감 이긴해요'
# 단어 변환
word_replace('  ',' ')
word_replace('이 니스프리','이니스프리')
word_replace('올 인원','올인원')
word_replace('에 탄올','에탄올')
word_replace('겅성','건성')
word_replace('쉐이빙 폼', '쉐이빙폼')
word_replace('저 작극', '저자극')
word_replace('지 복합성', '지복합성')
word_replace('펌 핑','펌핑')
word_replace('올이 원','올인원')
word_replace('피부트 러블','피부트러블')
word_replace('우르 오스','우르오스')
word_replace('평가 단','평가단')
word_replace('예 민성','예민성')
word_replace('파데','파운데이션')
word_replace('잔 여물','잔여물')
word_replace('올리브 영','올리브영')
word_replace('폼클렌징','클렌징폼')
word_replace('폼 클렌징','클렌징폼')
word_replace('클렌징 폼','클렌징폼')
word_replace('냄시','냄새')
word_replace('번 들','번들')
word_replace('글로 우픽','글로우픽')
word_replace('보송','뽀송')
word_replace('아재','아저씨')
word_replace('저 자극','저자극')
#word_replace('bb','비비')
#word_replace('BB','비비')
word_replace('비비','BB')
word_replace('bb','BB')
word_replace('뽀루지','뾰루지')
word_replace('피부가','피부')
word_replace('끈적','끈적 ')
#word_replace('남자친구','남친')
word_replace('향이','향')
word_replace('수부지','수분 부족형 지성 피부')
word_replace('프레시','프레쉬')
word_replace('남성','남자')
word_replace('민감성','민감')
word_replace('자극적','자극')
word_replace('번들','번들 ')
word_replace('여성','여자')
word_replace('아버지','아빠')
word_replace('아부지','아빠')
word_replace('지 성용','지성용')
word_replace('지 성','지성')
word_replace('선크림','썬크림')
word_replace('에 멀전','에멀전')
#word_replace('남친가','남친')
word_replace('끈덕','끈적')
word_replace('끈쩍','끈적')
word_replace('선물러','선물로')
word_replace('올인원','올인원 ')
word_replace('남치니','남자친구')
word_replace('남치 니','남자친구')
word_replace('남친','남자친구')
word_replace('화 하다고','화하다고')
word_replace('화 하게','화하게')
word_replace('화 하고','화하고')
word_replace('BRTC','비알티씨')
word_replace('비추천','비추')
word_replace('비 추천','비추')
word_replace('가스비','갸스비')
word_replace('민감','민감한 ')
word_replace('별론데','별로인데')
word_replace('별로 ','별로. ')
word_replace('에프터','애프터')
word_replace('클렌져','클랜저')
word_replace('별루','별로')
word_replace('피이지','PEG')
word_replace('바닐라 코','바닐라코')
word_replace('속당 김','속당김')
word_replace('XT M','XTM')
word_replace('피부','피부. ')
word_replace('피분데','피부인데')
word_replace('간 편','간편')
word_replace('간편','간편 ')
word_replace(' 리뷰',' .리뷰')
word_replace('에 멀젼','에멀젼')
word_replace('에멀전','에멀젼')
word_replace('뽀송뽀송','뽀송')
word_replace('푸석푸석','푸석')
word_replace('미끌미끌','미끌')
word_replace('뽀송뽀송','뽀득')
word_replace('뽀드득','뽀득')
word_replace('매끈매끈','매끈')
word_replace('보들','보들보들')

word_replace('DTRT','디티알티')

word_replace('  ',' ')

# 형태소 나누기

In [16]:
rew['morphs'] = rew['rew_pre'].map(k.morphs)

In [17]:
rew['pos'] = rew['rew_pre'].map(k.pos)

In [18]:
rew['rate'].value_counts()

good     2255
best     1504
soso      989
bad       354
worst     106
Name: rate, dtype: int64

In [19]:
rew_r = rew['rew_pre']
worst_r = rew.loc[rew['rate'] == 'worst']['rew_pre']
bad_r = rew.loc[rew['rate'] == 'bad']['rew_pre']
soso_r = rew.loc[rew['rate'] == 'soso']['rew_pre']
best_r = rew.loc[rew['rate'] == 'best']['rew_pre']
good_r = rew.loc[rew['rate'] == 'good']['rew_pre']
worst_r_a = ' '.join(worst_r)
bad_r_a = ' '.join(bad_r)
soso_r_a = ' '.join(soso_r)
best_r_a = ' '.join(best_r)
good_r_a = ' '.join(good_r)
rew_r_a = ' '.join(rew_r)

# 추가 전처리

In [20]:
r_copus = []
for i in rew['morphs']:
    for j in i:
        r_copus.append(j)

In [21]:
p_copus = []
for i in rew['pos']:
    for j in i:
        p_copus.append(j)

In [22]:
#p_copus

In [23]:
print('총 형태소 갯수 : {}'.format(len(r_copus)))

총 형태소 갯수 : 433482


In [24]:
count = Counter(r_copus)
words = dict(count.most_common())

In [25]:
count2 = Counter(p_copus)
words2 = dict(count2.most_common())

In [26]:
words

{'이': 16694,
 '하': 16358,
 '는': 10731,
 '.': 10271,
 'ㄴ': 9965,
 '고': 9851,
 '에': 6801,
 '은': 6748,
 '었': 5611,
 '도': 5535,
 '어': 5533,
 '좋': 5147,
 '쓰': 4791,
 '을': 4194,
 'ㄹ': 3846,
 '것': 3846,
 '가': 3693,
 '는데': 3608,
 '남자': 3453,
 '어서': 3349,
 '지': 3325,
 '같': 3157,
 '아': 3098,
 '게': 3076,
 '있': 3062,
 '어요': 2998,
 '향': 2976,
 '제품': 2904,
 '다': 2574,
 '않': 2571,
 '바르': 2551,
 '았': 2508,
 '피부': 2464,
 '나': 2451,
 '면': 2414,
 '되': 2387,
 'ㅂ니다': 2287,
 '더': 2146,
 '아요': 2116,
 '으로': 2116,
 '들': 2076,
 '주': 2062,
 '보': 1961,
 '거': 1923,
 '기': 1912,
 '잘': 1821,
 '습니다': 1787,
 '안': 1754,
 '라': 1698,
 '로': 1685,
 '시': 1677,
 '없': 1661,
 '느낌': 1564,
 '아서': 1544,
 '사용': 1544,
 '친구': 1481,
 '의': 1470,
 'ㅁ': 1454,
 '때': 1448,
 ',': 1422,
 '만': 1411,
 '저': 1403,
 '사': 1349,
 '네요': 1346,
 'ㄴ데': 1340,
 '음': 1317,
 '..': 1262,
 '지만': 1248,
 '를': 1245,
 '다고': 1196,
 '너무': 1120,
 '요': 1086,
 '이거': 1037,
 '좀': 1033,
 '보다': 1030,
 '냄새': 1027,
 '...': 1016,
 '분': 988,
 '니': 977,
 '아니': 977,
 '스킨': 935

In [27]:
noun = ['NN','NNG','NNB','NNM','NNP','NP','NR','ETN']
adj = ['VA','VXA','VC','VCN','VCP','VV','VXV','VX']
XR = ['XR']
V = ['VV','VXV','VX']

In [28]:
#words2

In [29]:
np.unique(np.array(list(words2.keys()))[:,1])

array(['ECD', 'ECE', 'ECS', 'EFA', 'EFI', 'EFN', 'EFO', 'EFQ', 'EPH',
       'EPT', 'ETD', 'ETN', 'IC', 'JC', 'JKC', 'JKG', 'JKI', 'JKM', 'JKO',
       'JKQ', 'JKS', 'JX', 'MAC', 'MAG', 'MDN', 'MDT', 'NNB', 'NNG',
       'NNM', 'NNP', 'NP', 'NR', 'OL', 'SE', 'SF', 'SP', 'SW', 'UN', 'VA',
       'VCN', 'VCP', 'VV', 'VX', 'VXA', 'VXV', 'XPN', 'XR', 'XSA', 'XSN',
       'XSV'], dtype='<U14')

In [30]:
a = pd.DataFrame(list(words2.keys()))
b = pd.DataFrame(list(words2.values()))
c = pd.merge(a,b,how='outer',left_index = True, right_index = True)
c.columns = ['word','pos','count']

In [31]:
noun = ['NNG','NNB','NNM','NNP','NP','NR','ETN']

## 명사

- 안 : 부정임을 나타내기 위해 남겨둠
- 선크림을 사전에 추가해도 선 + 크림으로 나눠져서 (선크림 -> 썬크림 변환)
- 올,인원 : 올인원이 주변에 어떤 단어가 오느냐에 따라 올인원, 올/인원으로 나눠져서 남겨둠.
- 당,김 : 당김이 추가가 안되서 남겨둠

In [32]:
noun = ['NNG','MMB','NNM','NNP','NP','NR','UN']

In [33]:
df_n = c.loc[(c['pos']=='NNG')|(c['pos']=='NNB')|(c['pos']=='NNM')|(c['pos']=='NNP')|(c['pos']=='NP')|(c['pos']=='NR')|(c['pos']=='UN')]

In [34]:
df_n.iloc[249:,:]

,word,pos,count
609,경우,NNG,89
610,베이스,NNG,89
612,돈,NNG,88
613,일,NNM,88
614,바,NNG,88
615,눈,NNG,88
616,이것저것,NNG,88
618,김,NNG,87
619,구,NR,87
620,매트,NNG,87


In [35]:
#noun_stop_words = ['것','거','때','저','이거','수','분','나','듯','정도','편','이것','후','전','하나','임','중','점','라','감','뭐','데','제가','때문','이','여','번','부분','무','남','건','유','하다','비','감이','함','개','해','원래']
noun_stop_words = ['것','거','때','저','이거','수','분','나','듯','정도','편','이것','후','전','하나','임','중','점','라','감','뭐','데','제가','때문','이','여','번','부분','무','남','건','유','하다','비','감이','함','개','해','원래','날','고','자체','있음','리','별','말','짱','무엇','가','꺼','내','쓰기','줄','시','차','내가','우리','짐','일','저희','진짜','터','시','인','이번','간','앞','인','얘','뒤','시','요','위','니','구','신','거림','니','쪽']

In [36]:
'대' in noun_stop_words

False

In [37]:
df_n.loc[df_n['word'] == '아해']

,word,pos,count
4100,아해,NNG,3


In [38]:
rew['rew_pre'].iloc[find_pos_idx(('니', 'NP'))].iloc[3]

'친구가 쓰는데 내가 늘말하지만 걔 피부. 는 벽지처럼 하얗다 하얀 니가 부럽다 임마'

In [39]:
rew['pos'].iloc[find_pos_idx(('쪽', 'NNB'))].iloc[4]

[('화장품', 'NNG'),
 ('들', 'XSN'),
 ('이', 'JKS'),
 ('다', 'MAG'),
 ('떨어지', 'VV'),
 ('어서', 'ECD'),
 ('얼마', 'NNG'),
 ('전', 'NNG'),
 ('약국', 'NNG'),
 ('가', 'VV'),
 ('아서', 'ECD'),
 ('필요', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('거', 'NNB'),
 ('다', 'MAG'),
 ('사', 'VV'),
 ('아', 'ECS'),
 ('오', 'VX'),
 ('았', 'EPT'),
 ('는데', 'ECD'),
 ('그', 'MDT'),
 ('때', 'NNG'),
 ('이것', 'NP'),
 ('도', 'JX'),
 ('사', 'VV'),
 ('었', 'EPT'),
 ('어요', 'EFN'),
 ('.', 'SF'),
 ('여드름', 'NNG'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('나', 'JX'),
 ('각질', 'NNG'),
 ('잘', 'MAG'),
 ('생기', 'VV'),
 ('는', 'ETD'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('용', 'NNG'),
 ('화장품', 'NNG'),
 ('은', 'JX'),
 ('보습', 'NNG'),
 ('이', 'JKS'),
 ('잘', 'MAG'),
 ('안', 'MAG'),
 ('되', 'VV'),
 ('는', 'ETD'),
 ('편', 'NNB'),
 ('이', 'VCP'),
 ('ㄴ데', 'ECE'),
 ('오히려', 'MAG'),
 ('보습', 'NNG'),
 ('을', 'JKO'),
 ('안', 'MAG'),
 ('하', 'VV'),
 ('어', 'ECS'),
 ('주', 'VXV'),
 ('면', 'ECE'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('드', 'VV'),
 ('어', 'ECS'),
 ('안', 'MAG'),
 ('좋아지', 'VV'),
 ('거든요', 'EFN')

In [40]:
len(rew['rew_pre'].iloc[find_word_sen('남치 니')])

0

In [41]:
k.pos("20")

[('20', 'NR')]

## 동사

In [42]:
verb = ['VV','VXV','VCP']

In [43]:
v = c.loc[(c['pos']=='VV')|(c['pos']=='VXV')|(c['pos']=='VCP')]
v

,word,pos,count
3,이,VCP,7896
11,쓰,VV,4789
12,하,VV,4776
31,바르,VV,2502
34,않,VXV,2294
39,있,VV,2013
46,주,VXV,1664
54,되,VV,1455
63,사,VV,1281
81,있,VXV,1015


In [44]:
# 넣을 것만 고른 것
ver = ['바르','발르','않','좋아하','싫어하','않','좋아지','귀찮','안하','땡기','미끌거리','뒤집','하얘지','못쓰','어울리','밝아지','지우','올라오','강하','발라보','씻','묻','씻기','뭉치','일어나','싸라지']

In [45]:
k.pos('씻을때 좋은')

[('씻', 'VV'), ('을', 'ETD'), ('때', 'NNG'), ('좋', 'VA'), ('은', 'ETD')]

In [46]:
rew['rew_pre'].iloc[find_pos_idx(('이', 'VCP'))].iloc[0]

'향 청량해서 좋아요. 흡수가 잘되면서 끈적 이지 않아요. 지금 시즌 잘 사용중인데 가을부터는 건조할 듯 합니다. 가볍게 쓰기 좋대요.'

In [47]:
rew['pos'].iloc[find_pos_idx(('아', 'VV'))].iloc[2]

[('좋', 'VA'),
 ('기', 'ETN'),
 ('는', 'JKS'),
 ('좋', 'VA'),
 ('은데', 'ECD'),
 ('솔직히', 'MAG'),
 ('이', 'MDT'),
 ('가격', 'NNG'),
 ('주고', 'NNG'),
 ('사', 'VV'),
 ('ㄹ', 'ETD'),
 ('정도', 'NNG'),
 ('는', 'JX'),
 ('아니', 'VCN'),
 ('에요', 'EFN'),
 ('.', 'SF'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('에', 'JKM'),
 ('드라마', 'NNG'),
 ('틱하', 'XSA'),
 ('ㄴ', 'ETD'),
 ('효과', 'NNG'),
 ('가', 'JKS'),
 ('보이', 'VV'),
 ('는', 'ETD'),
 ('정도', 'NNG'),
 ('도', 'JX'),
 ('아니', 'VV'),
 ('고', 'ECE'),
 ('백화점', 'NNG'),
 ('이랑', 'JC'),
 ('로드', 'NNG'),
 ('샵', 'NNG'),
 (',', 'SP'),
 ('드럭', 'UN'),
 ('스토어', 'NNG'),
 ('에', 'JKM'),
 ('도', 'JX'),
 ('이', 'MDT'),
 ('정도', 'NNG'),
 ('혹은', 'MAG'),
 ('이것', 'NP'),
 ('보다', 'JKM'),
 ('좋', 'VA'),
 ('은', 'ETD'),
 ('토너', 'NNG'),
 ('들', 'XSN'),
 ('있', 'VV'),
 ('고', 'ECE'),
 ('아무리', 'MAG'),
 ('비싸도', 'UN'),
 ('가격', 'NNG'),
 ('이', 'JKS'),
 ('이렇', 'VA'),
 ('게', 'ECD'),
 ('는', 'JX'),
 ('아', 'VV'),
 ('아', 'ECS'),
 ('뉘', 'VV'),
 ('ㄴ', 'ETD'),
 ('데', 'NNB'),
 ('....', 'SW'),
 ('가격', 'NNG'),
 ('에', 'JKM'),
 ('늘', 'V

## 형용사

In [48]:
ad = ['VA','VXA','VCN']

In [49]:
adj = c.loc[(c['pos']=='VA')|(c['pos']=='VXA')|(c['pos']=='VCN')]
adj

,word,pos,count
10,좋,VA,5118
23,같,VA,3140
47,없,VA,1648
97,괜찮,VA,837
118,아니,VCN,614
125,그렇,VA,566
130,많,VA,525
141,크,VA,466
172,가볍,VA,372
173,싶,VXA,372


In [50]:
adj.loc[adj['word']=='궁금하']

,word,pos,count
1447,궁금하,VA,24


In [51]:
adj_words = ['같','그렇','싶','그','길','만하','이렇','어떻','바르','있','계시','궁금하']

In [52]:
rew['rew_pre'].iloc[find_pos_idx(('똑같', 'VA'))].iloc[2]

'남자친구이 뾰루지 올라와서 면세찬스로 티쀼르 세트 사줌 증정용으로 따라옴 냄새는 남자스킨 냄새 내 코엔 다 똑같음 효과는 잘 모르겟음 쿨링 느낌이 강함'

In [53]:
rew['pos'].iloc[find_pos_idx(('굳', 'VA'))].iloc[1]

[('역시', 'MAG'),
 ('스킨', 'NNG'),
 ('푸드', 'NNG'),
 ('는', 'JX'),
 ('가격', 'NNG'),
 ('대비', 'NNG'),
 ('기능', 'NNG'),
 ('이', 'JKS'),
 ('좋', 'VA'),
 ('아요', 'EFN'),
 ('가성비', 'NNG'),
 ('는', 'JX'),
 ('굳', 'VA')]

## 부사

In [54]:
m = ['MAG','MAC']

In [55]:
mag = ['뽀송뽀송','끈적','뽀득','촉촉','보들보들','부들부들','깔끔','뻑뻑','산뜻','딱딱','푸석푸석','미끌미끌','뽀드득','뽀득뽀득','울긋불긋','매끈매끈']

In [56]:
ma = c.loc[(c['pos']=='MAG')|(c['pos']=='MAC')]
ma

,word,pos,count
43,잘,MAG,1818
61,안,MAG,1345
73,너무,MAG,1119
77,좀,MAG,1025
92,더,MAG,893
100,다,MAG,816
106,많이,MAG,747
121,그냥,MAG,598
139,일단,MAG,476
140,정말,MAG,474


In [57]:
rew['rew_pre'].iloc[find_pos_idx(('땡', 'MAG'))].iloc[0]

'단종 아닙니다. 정정하세요. 여친님께서 선물해주신 제품입니다. 원래 쓰던 올인원 제품이 단종 돼 버려서 대체재를 잘 못 찾고 있던 찰나에 선물 받게 됐습니다. 일단 제형 자체는 크리미한 편입니다. 이전까지 써오던 올인원 제품들 중 제일 진합니다. 펌핑 하면서 생각보다 진하길래 바르고 나서 유분기가 많이 나오면 어쩌나 싶었는데 그다지 심하지 않습니다. 흡수력은 빠른 편이고 트러블 진정효과가 있네요. 지금 러쉬팩이랑 같이 쓰고 있어서 이 제품만의 피부. 진정 효과가 어느 정도인지는 정확하게 가늠이 가지는 않으나, 확실히 느껴지기는 합니다. 보습력도 좋고 향도 좋고 다 좋습니다. 여친님 쵝오 . 다만 피지조절 효과는 아직 큰 덕은 못 보고 있습니다. 애초에 피부. 전체가 거의 중동 지역 산유국 이상의 기름을 보유하고 있습니다. 중동, 미국, 러시아 등 각국에 매장된 기름은 언젠가는 동나기 마련이지만 제 피부. 는 그렇지 않습니다. 자고 일어나면 나오고, 오후만 지나면 나오고, 저녁 먹을 때 즈음 또 나오는 게 거의 무한 동력기와 다르지 않습니다. 이런 극 악조건인 피부. 에게 이 제품 하나로 뽀송한 피부. 를 바라는 건 말도 안 되는 일이겠지요. 피지 조절 효과가 크다고는 말씀드리기 어려우나 이 정도면 괜찮은 편이라고 말하고 싶습니다. 선물이란게 참 그렇습니다. 선물 그 자체가 나에게 주는 효용보다도, 선물을 준비한 사람의 마음이 나에게 전해질 때 느껴지는 그 감정이란게 참 좋은 거 같습니다. 이른 아침 백화점에 뛰어가서 이 제품을 골라 왔을 여친님 생각에 입가에 미소가 번집니다. 여친님 땡큐.'

In [58]:
rew['pos'].iloc[find_pos_idx(('땡', 'MAG'))].iloc[1]

[('가격', 'NNG'),
 ('이', 'JKS'),
 ('혜', 'UN'),
 ('자', 'NNG'),
 ('스럽', 'XSA'),
 ('어서', 'ECD'),
 ('남편', 'NNG'),
 ('에게', 'JKM'),
 ('츤데레', 'UN'),
 ('로', 'JKM'),
 ('선물', 'NNG'),
 ('하', 'XSV'),
 ('어', 'ECS'),
 ('주기', 'NNG'),
 ('좋앗', 'UN'),
 ('음', 'NNG'),
 ('28000', 'NR'),
 ('원', 'NNM'),
 ('에', 'JKM'),
 ('1', 'NR'),
 ('1', 'NR'),
 ('이', 'VCP'),
 ('라서', 'ECD'),
 ('가성비', 'NNG'),
 ('갑', 'NNG'),
 ('토너', 'NNG'),
 ('이', 'VCP'),
 ('ㄴ데', 'ECE'),
 ('도', 'JX'),
 ('올', 'NNG'),
 ('인원', 'NNG'),
 ('처럼', 'JKM'),
 ('촉촉', 'XR'),
 ('하', 'XSA'),
 ('어서', 'ECD'),
 ('여름', 'NNG'),
 ('에', 'JKM'),
 ('는', 'JX'),
 ('이거', 'NP'),
 ('하나', 'NR'),
 ('만', 'JX'),
 ('바르', 'VV'),
 ('면', 'ECE'),
 ('말', 'VXV'),
 ('ㄴ', 'ETD'),
 ('사', 'NNG'),
 ('땡', 'MAG'),
 ('일', 'VV'),
 ('듯', 'ECD'),
 ('향', 'NNG'),
 ('도', 'JX'),
 ('나름', 'NNG'),
 ('은', 'JX'),
 ('은', 'JX')]

## 관형사

In [59]:
md = c.loc[(c['pos']=='MDT')|(c['pos']=='MDN')]
md

,word,pos,count
74,이,MDT,1093
165,한,MDN,388
196,그,MDT,347
205,다른,MDT,332
296,몇,MDT,204
337,두,MDN,177
411,여러,MDT,144
423,그런,MDT,140
516,어느,MDT,110
538,세,MDN,105


In [60]:
rew['rew_pre'].iloc[find_pos_idx(('다른', 'MDT'))].iloc[1]

'이거 파란색도 잇음. 이름은 아쿠 아파워 똑같은데 통이 파란색임. 근데 말야 이거 여름에 냉장고에 넣어두고 쓰면 스킨브레이서 만큼 얼굴 소독되고 시원한데 너무 얼굴에서 금방 마르니까 양을 듬뿍 써야 함. 생각해보면 200ML고 성분이 젤이라서 물타입 액체보다 많은 양이 필요한데 엄청 비쌈. 54000원은 아니고 더 싸게는 살수 있지만 무슨 부자집 도련님 스킨 같음. 너무 비쌈. 이거 좋긴 한데 월급이 300만원 이 하이면 안쓰는게 나음. 가격 성분이나 원가에 비해 너무 비쌈. 취향을 존중하므로 월급이 적더라도 사용하실 분은 사용해도 됨. 비싸서 탈락 그리고 성분을 보면 가격에 비해 좋은 성분이 아니라 알러지 유발 성분 등 안좋은 게 있어서 가격대비 완전 탈락 이니스프리가 요즘 유해성분이 거의 아주 없는 것들 만드는데 다른 화장품에 비해 가격대비 제일 좋음. 이니스프리 광고하는 사람 아님 진실을 말하는 자임'

In [61]:
rew['pos'].iloc[find_pos_idx(('여러', 'MDT'))].iloc[1]

[('급하', 'VA'),
 ('게', 'ECD'),
 ('스킨', 'NNG'),
 ('케어', 'NNG'),
 ('제품', 'NNG'),
 ('사', 'NNG'),
 ('라고', 'JX'),
 ('하면', 'NNP'),
 ('살만', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('이니스프리', 'NNG'),
 ('포', 'NNG'),
 ('맨', 'NNG'),
 ('라인', 'NNG'),
 ('.', 'SF'),
 ('여러', 'MDT'),
 ('기능성', 'NNG'),
 (',', 'SP'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('타입', 'NNG'),
 ('별로', 'MAG'),
 ('.', 'SF'),
 ('출시', 'NNG'),
 ('되', 'XSV'),
 ('어', 'ECD'),
 ('있', 'VXV'),
 ('기', 'ETN'),
 ('는', 'JKS'),
 ('하', 'VV'),
 ('지만', 'ECE'),
 ('전체적', 'NNG'),
 ('으로', 'JKM'),
 ('크', 'VA'),
 ('게', 'ECD'),
 ('다른', 'MDT'),
 ('점', 'NNG'),
 ('은', 'JX'),
 ('못', 'MAG'),
 ('느끼', 'VV'),
 ('겠', 'EPT'),
 ('고', 'ECE'),
 ('그냥', 'MAG'),
 ('순하', 'VA'),
 ('고', 'ECE'),
 ('무난', 'XR'),
 ('하', 'XSA'),
 ('다', 'EFN'),
 ('.', 'SF'),
 ('피부', 'NNG'),
 ('.', 'SF'),
 ('에', 'JKM'),
 ('크', 'VA'),
 ('ㄴ', 'ETD'),
 ('자극', 'NNG'),
 ('없', 'VA'),
 ('고', 'ECE'),
 ('크', 'VA'),
 ('ㄴ', 'ETD'),
 ('효과', 'NNG'),
 ('도', 'JX'),
 ('없', 'VA'),
 ('다', 'EFN'),
 ('.', 'SF'),
 ('급', 'NNG'),

## 어근

- 사용할 것

In [62]:
xr = c.loc[c['pos'] == 'XR']
xr

,word,pos,count
98,촉촉,XR,830
144,무난,XR,454
236,민감,XR,279
237,산뜻,XR,279
245,시원,XR,266
266,은은,XR,242
328,뽀송,XR,184
330,깔끔,XR,182
437,적당,XR,136
483,저렴,XR,118


In [63]:
xr.loc[xr['word'] == '시원']

,word,pos,count
245,시원,XR,266


- 뿌리(어떻게 사용되는지 다시 한번 확인. 1. 뿌리다, 2. 머리 뿌리)
- 유(유수분 남겨두기)
- 저(저자극/ 지우지말기)
- 아이크림(사전추가할지말지)
- 남성용

In [64]:
#noun = ['NN','NNG','NNB','NNM','NNP','NP','NR','ETN']
#adj = ['VA','VXA','VC','VCN','VCP','VV','VXV','VX']
#XR = ['XR']
#V = ['VV','VXV','VX']

## 외국어

- 사용할 것

In [65]:
c.loc[c['pos'] == 'OL']

,word,pos,count
164,BB,OL,392
818,ML,OL,58
1280,X,OL,29
1380,XTM,OL,26
1608,T,OL,20
1659,UHD,OL,20
1693,CC,OL,19
1736,PEG,OL,18
1760,M,OL,18
2044,N,OL,13


## 명사 파생 접미사

In [66]:
c.loc[(c['pos']=='XSN')]

,word,pos,count
45,들,XSN,1705
101,력,XSN,791
145,성,XSN,454
206,용,XSN,331
260,감,XSN,252
611,째,XSN,88
637,형,XSN,83
657,적,XSN,79
674,씩,XSN,76
776,로,XSN,63


In [67]:
rew['rew_pre'].iloc[find_pos_idx(('들', 'XSN'))].iloc[1]

'아빠 사용하시는 제품인데 겨울에 스킨과 로션만 발라도 충분하다고 좋아하십니다 원래 제가 준 화장품 주로 사용하시는 편인데 이것도 좋다고 나중에 세일하면 또 사달라고 하시네요 향은 아빠들 화장품 향... 이고 트러블도 없고 가격도 세일하면 싸게 구입할 수도 있어서 잘 쓰시는 제품입니다'

In [68]:
rew['pos'].iloc[find_pos_idx(('째', 'XSN'))].iloc[13]

[('아빠', 'NNG'),
 ('선물', 'NNG'),
 ('로', 'JKM'),
 ('드리', 'VV'),
 ('었', 'EPT'),
 ('고', 'ECE'),
 ('좋', 'VA'),
 ('아', 'ECD'),
 ('허심', 'NNG'),
 ('2', 'NR'),
 ('년', 'NNG'),
 ('째', 'XSN'),
 ('쓰', 'VV'),
 ('시', 'EPH'),
 ('는', 'ETD'),
 ('제품', 'NNG'),
 ('순하', 'VA'),
 ('어요', 'EFN')]

## 체언 접두사

- 사용

In [69]:
rew['pos'].iloc[find_pos_idx(('무', 'XPN'))].iloc[13]

[('평소', 'NNG'),
 ('에', 'JKM'),
 ('화장품', 'NNG'),
 ('은', 'JX'),
 ('썬크림', 'NNG'),
 ('말', 'VV'),
 ('곤', 'ECS'),
 ('다', 'MAG'),
 ('필요없', 'VA'),
 ('어', 'ECD'),
 ('하', 'VV'),
 ('시', 'EPH'),
 ('는', 'ETD'),
 ('아빠', 'NNG'),
 ('때문', 'NNB'),
 ('에', 'JKM'),
 ('남자', 'NNG'),
 ('올인원', 'NNG'),
 ('제품', 'NNG'),
 ('을', 'JKO'),
 ('알아보', 'VV'),
 ('더', 'EPT'),
 ('ㄴ', 'ETD'),
 ('중', 'NNG'),
 ('평가단', 'NNG'),
 ('을', 'JKO'),
 ('하', 'VV'),
 ('게', 'ECD'),
 ('되', 'VV'),
 ('었', 'EPT'),
 ('네요', 'EFN'),
 ('썬크림', 'NNG'),
 ('말', 'VV'),
 ('고는', 'ECE'),
 ('필요', 'NNG'),
 ('없', 'VA'),
 ('다는', 'ETD'),
 ('우리', 'NP'),
 ('아빠', 'NNG'),
 ('도', 'JX'),
 ('잘', 'MAG'),
 ('챙기', 'VV'),
 ('어', 'ECS'),
 ('바르', 'VV'),
 ('시', 'EPH'),
 ('는', 'ETD'),
 ('것', 'NNB'),
 ('을', 'JKO'),
 ('보', 'VV'),
 ('니', 'ECD'),
 ('올', 'NNG'),
 ('인원', 'NNG'),
 ('이', 'VCP'),
 ('라', 'ECD'),
 ('역시', 'MAG'),
 ('편하', 'VA'),
 ('기', 'ETN'),
 ('는', 'JKS'),
 ('편하', 'VA'),
 ('ㅂ니다', 'EFN'),
 ('제가', 'NNG'),
 ('테스트', 'NNG'),
 ('해보', 'VV'),
 ('니', 'ECD'),
 ('마무리', 'NNG'),
 ('

In [70]:
c.loc[(c['pos']=='XPN')]

,word,pos,count
834,무,XPN,56
1052,친,XPN,39
1603,부,XPN,20
1803,제,XPN,17
2301,잡,XPN,11
2356,건,XPN,10
2589,저,XPN,8
2607,드,XPN,8
2816,신,XPN,7
2951,초,XPN,7


# 불용어 제거

In [71]:
pos = ['NNG','MMB','NNM','NNP','NP','NR','UN','VA','VXA','VCN','XR','OL','XPN']
in_pos = ['VV','VXV','VCP','MAG','MAC']

In [72]:
stop_words = ['것','거','때','저','이거','수','분','나','듯','정도','편','이것','후','전','하나','임','중','점','라','감','뭐','데','제가','때문','이','여','번','부분','무','남','건','유','하다','비','감이','함','개','해','원래','날','고','자체','있음','리','별','말','짱','무엇','가','꺼','내','쓰기','줄','시','차','내가','우리','짐','일','저희','진짜','터','시','인','이번','간','앞','인','얘','뒤','시','요','위','니','구','신','거림','니','쪽','같','그렇','싶','그','길','만하','이렇','어떻','바르','있','계시','궁금하']
words = ['바르','발르','않','좋아하','싫어하','않','좋아지','귀찮','안하','땡기','미끌거리','뒤집','하얘지','못쓰','어울리','밝아지','지우','올라오','강하','발라보','씻','묻','씻기','뭉치','일어나','싸라지','뽀송뽀송','끈적','뽀득','촉촉','보들보들','부들부들','깔끔','뻑뻑','산뜻','딱딱','푸석푸석','미끌미끌','뽀드득','뽀득뽀득','울긋불긋','매끈매끈']

In [73]:
rew['final'] = rew['pos'].map(lambda x: [i[0] for i in x if ((i[1] in pos)&(i[0] not in stop_words))|(i[1] in in_pos)&(i[0] in words)])

# 임베딩

In [106]:
rew['token_len'] = rew['final'].map(lambda x: len(x))

In [115]:
rew['token_len'].value_counts()

9      234
8      221
10     206
6      198
7      197
4      195
11     185
5      180
13     175
15     173
12     161
14     156
17     149
16     147
3      129
18     122
19     111
20     104
21     103
24      91
22      90
26      74
23      68
27      67
2       66
25      66
30      62
28      55
31      49
29      49
37      49
36      48
34      47
32      46
33      45
41      44
42      42
39      37
38      35
35      34
46      34
52      29
40      29
47      27
45      26
51      24
49      24
44      23
50      23
43      23
48      23
53      22
1       22
60      21
54      21
57      20
55      20
63      19
64      19
65      17
56      17
61      17
58      16
59      15
62      14
71      13
67      13
73      12
68      12
80      10
77      10
88       9
82       9
72       9
92       8
81       8
84       8
74       8
99       8
70       8
66       7
90       7
76       7
89       7
93       7
100      6
75       6
118      6
97       6
69       5
85       5

In [118]:
rew.loc[rew['token_len'] == 2]

,date,user_id,sex,rate,content,product_url,age,skin_type,rew_pre,morphs,pos,final,embedding,token_len
2,3개월 전,minseo6666,f,good,아빠 사드렸는데 저아해여,/product/7212,17세,복합성,아빠 사드렸는데 저아해여,"[아빠, 사, 아, 드리, 었, 는데, 저, 아해, 열]","[(아빠, NNG), (사, VV), (아, ECS), (드리, VXV), (었, ...","[아빠, 아해]","[0.001658663, 0.005998151, -0.0056231045, 0.03...",2
107,2018.2.8,새우청년,m,good,전 좀 끈적였는데 자극은 덜 하긴 했어요,/product/4108,38세,건성,전 좀 끈적 였는데 자극은 덜 하긴 했어요,"[전, 좀, 끈적, 이, 었, 는데, 자극, 은, 덜, 하, 기, 는, 하, 었, 어요]","[(전, NNG), (좀, MAG), (끈적, NNG), (이, VV), (었, E...","[끈적, 자극]","[0.013146712, 0.020671917, -0.036023855, 0.021...",2
129,2016.3.22,kioo1300,m,best,비싸지만 않으면 !!,/product/4108,30세,건성,비싸지만 않으면,"[비싸, 지만, 않, 으면]","[(비싸, VA), (지만, ECE), (않, VXV), (으면, ECD)]","[비싸, 않]","[0.0252061, 0.027127434, -0.048106916, 0.00881...",2
133,2016.1.15,세류.,m,good,역시 비오템... 흡수가 잘되는듯,/product/4108,25세,건성,역시 비오템... 흡수가 잘되는듯,"[역시, 비오템, ..., 흡수, 가, 잘되, 는, 듯]","[(역시, MAG), (비오템, NNP), (..., SE), (흡수, NNG), ...","[비오템, 흡수]","[0.025747318, 0.020784337, -0.017592188, 0.040...",2
217,2019.3.28,kimki8,f,best,여름이 다 왔다니까 시원해요,/product/11071,21세,건성,여름이 다 왔다니까 시원해요,"[여름, 이, 다, 오, 았, 다니까, 시원, 하, 어요]","[(여름, NNG), (이, JKS), (다, MAG), (오, VV), (았, E...","[여름, 시원]","[0.019420141, 0.03550126, -0.0324269, 0.035216...",2
266,2016.11.1,한국인좀비,f,best,귀찮을땐 올인원이 짱이지!!!,/product/21577,4세,민감성,귀찮을 땐 올인원 이 짱이지,"[귀찮, 을, 때, ㄴ, 올인원, 이, 짱, 이, 짓]","[(귀찮, VA), (을, ETD), (때, VV), (ㄴ, ETD), (올인원, ...","[귀찮, 올인원]","[0.03168128, 0.08438602, -0.04123715, 0.072067...",2
279,2019.2.20,blakeleee,m,best,산뜻함과 함께 진정까지,/product/13159,34세,복합성,산뜻함과 함께 진정까지,"[산뜻, 하, ㅁ, 과, 함께, 진정, 까지]","[(산뜻, XR), (하, XSA), (ㅁ, ETN), (과, JKM), (함께, ...","[산뜻, 진정]","[0.025245309, 0.024438767, -0.03518594, 0.0230...",2
287,2018.9.25,앨리스1234,m,best,저는 진짜 잘 맞아서 계속 쓸 생각이 있는 제품입니다.,/product/13159,27세,건성,저는 진짜 잘 맞아서 계속 쓸 생각이 있는 제품입니다.,"[저, 는, 진짜, 잘, 맞, 아서, 계속, 쓰, ㄹ, 생각, 이, 있, 는, 제품...","[(저, NP), (는, JX), (진짜, MAG), (잘, MAG), (맞, VV...","[생각, 제품]","[0.021658752, 0.026687304, -0.022809055, 0.014...",2
340,2017.11.16,솔직한,f,best,아빠한테 사줬어요! 되게 좋아하시더라구요ㅎㅎ,/product/7198,21세,복합성,아빠한테 사줬어요 되게 좋아하시더라구요,"[아빠, 한테, 사, 아, 주, 었, 어요, 되게, 좋아하, 시, 더, 라구요]","[(아빠, NNG), (한테, JKO), (사, VV), (아, ECS), (주, ...","[아빠, 좋아하]","[0.015888134, 0.020914, -0.026845822, 0.073089...",2
362,2015.4.22,주넌,m,bad,바른 듯 안 바른 듯,/product/7198,29세,지성,바른 듯 안 바른 듯,"[바르, ㄴ, 듯, 안, 바르, ㄴ, 듯]","[(바르, VV), (ㄴ, ETD), (듯, NNB), (안, MAG), (바르, ...","[바르, 바르]","[0.034573354, 0.02898158, -0.012718668, 0.0703...",2


In [74]:
cps = []
for i in rew['final']:
    cps.append(i)

In [167]:
cps

[['무난', '가볍', '좋', '년'],
 ['남자', '친구', '선물', '나쁘', '좋', '않'],
 ['아빠', '아해'],
 ['향', '좋', '피부', '민감', '오더'],
 ['가격', '저렴', '냄새', '쎄', '좋', '다음', '구매'],
 ['선물', '사서', '선물', '만족', '바르'],
 ['향',
  '청량',
  '좋',
  '흡수',
  '끈적',
  '않',
  '지금',
  '시즌',
  '사용',
  '가을',
  '부',
  '건조',
  '가볍',
  '좋'],
 ['피부',
  '지성',
  '최대한',
  '안',
  '끈적',
  '거리',
  '수분',
  '보충',
  '제품',
  '좋',
  '피부',
  '기름',
  '클렌징',
  '폼',
  '세안',
  '바르',
  '괜찮'],
 ['남자',
  '친구',
  '사',
  '줫',
  '좋',
  '남자',
  '무겁',
  '싫어하',
  '텍스',
  '좋',
  '수분',
  '감도',
  '좋앗',
  '향',
  '나쁘',
  '않',
  '가격대',
  '용량',
  '나쁘',
  '않'],
 ['남자', '화장품', '향', '중시', '아빠', '선물', '만족', '두통'],
 ['지금', '오빠', '트러블', '없', '향기', '도', '괜찮', '애용', '제품'],
 ['복합',
  '피부',
  '생각',
  '피부',
  '결',
  '않',
  '생각',
  '간지럽',
  '피부',
  '이상',
  '수도',
  '빨갛',
  '올라오'],
 ['실랑이', '복합', '좋', '주변', '얼굴색', '좋아지', '좋', '굿'],
 ['예민', '분듵', '트러블'],
 ['아빠', '생신', '선물', '향', '좋', '아빠', '완전', '만족', '트러블', '무난', '선물', '좋'],
 ['가볍', '바르', '좋', '향', '강하', '않', '아빠', '2', '통', '무난', '

In [75]:
from gensim.models import FastText

In [147]:
ft_model = FastText(rew['final'], size=100, window=5, min_count=1, workers=4, sg=1)
ft_model5 = FastText(rew['final'], size=100, window=5, min_count=3, workers=4, sg=1)

In [174]:
ft_cps = FastText(cps, size=100, window=5, min_count=3, workers=4, sg=1)
ft_cps1 = FastText(cps, size=100, window=5, min_count=1, workers=4, sg=1)

- windos : window 갯수
- min_count : 최소한 토큰 갯수(min_count 밑은 무시)
- sg : Training algorithm: skip-gram if sg=1, otherwise CBOW.
- workers : 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

In [185]:
rew['embedding'] = rew['final'].map(lambda x: ft_cps[' '.join(x)])

In [183]:
ft_cps.most_similar('아빠',topn = 10)

[('생신', 0.9081434011459351),
 ('엄마', 0.8794147372245789),
 ('하심', 0.8769795894622803),
 ('동생', 0.8563430309295654),
 ('남동생', 0.8534249663352966),
 ('뿌듯', 0.8442403078079224),
 ('친오빠', 0.8383175134658813),
 ('삼촌', 0.834470272064209),
 ('오빠', 0.8165725469589233),
 ('덕분', 0.8077235221862793)]

In [184]:
ft_cps1.most_similar('아빠',topn = 10)

[('하심', 0.9425002336502075),
 ('남동생', 0.9162098169326782),
 ('생신', 0.9107539653778076),
 ('동생', 0.906241774559021),
 ('엄마', 0.9025672674179077),
 ('친오빠', 0.8969951868057251),
 ('뿌듯', 0.8958461284637451),
 ('계세', 0.8866431713104248),
 ('줫', 0.8775418996810913),
 ('보시', 0.877208948135376)]

In [166]:
rew[['final','embedding']]['embedding'][0]

array([-8.6372842e-05, -3.1053837e-04, -2.3039342e-03, -2.4283952e-04,
       -7.6438772e-04,  1.9748993e-03, -5.2524882e-04,  7.9962431e-04,
       -6.6049804e-04, -9.6846244e-04, -5.8824196e-04, -1.1744833e-03,
        4.8566441e-04, -1.8489932e-03,  1.0870718e-03,  2.0836582e-04,
       -8.5990195e-04, -4.8440570e-06, -1.2837243e-03, -2.3418337e-03,
        1.3438982e-04,  7.9294055e-04, -8.7414948e-05,  1.2354492e-04,
       -7.7872234e-04,  1.1222021e-03,  2.0759392e-03, -2.4502078e-04,
       -3.9810408e-04, -1.3049669e-04,  5.3364882e-04, -2.0091871e-03,
       -4.8729591e-04,  9.1204216e-05,  9.0011209e-04, -6.1242882e-04,
       -1.2414425e-03,  5.1010679e-04, -5.5256800e-04, -1.4216008e-03,
        3.5252766e-04,  7.5422613e-05,  1.5012338e-03, -1.0779711e-03,
       -1.2339543e-05, -2.7158675e-03, -6.5032457e-04,  1.5060044e-03,
        1.3155012e-03, -5.0944946e-04,  1.9970584e-04,  6.4714276e-04,
        1.2997796e-03,  7.9203688e-04,  9.5222594e-04, -1.4666071e-03,
      

In [188]:
rew['final'][0]

['무난', '가볍', '좋', '년']

In [191]:
ft_cps[rew['final'][0]]

array([[ 0.3609701 , -0.05689764, -0.04849442,  0.17830601, -0.18166591,
        -0.16129665, -0.25392008, -0.2755426 , -0.10342938, -0.0768807 ,
        -0.15377483, -0.15054774,  0.09471117, -0.1237134 , -0.12823829,
         0.18315297, -0.08369831, -0.06515842, -0.03965817,  0.03223301,
        -0.34407637,  0.20852259, -0.03102021, -0.08645078, -0.1018407 ,
         0.08058178, -0.03913326, -0.01458384, -0.2860216 ,  0.2259741 ,
        -0.06371759, -0.15293142, -0.22524542, -0.01884172,  0.17963955,
        -0.23536734,  0.24521624, -0.07164592,  0.24989659,  0.07794171,
         0.2937536 , -0.01453943, -0.47106722, -0.04449168, -0.17849563,
         0.14539218,  0.04096899,  0.04825764,  0.18451995, -0.22116904,
        -0.26141128, -0.06801713, -0.26395282,  0.14675897,  0.04945637,
        -0.0777038 ,  0.01333133, -0.17366749, -0.20608664,  0.21068114,
        -0.04297358,  0.34747908,  0.25212497,  0.12133831, -0.07209365,
        -0.37588555, -0.12723826, -0.02986211,  0.1

In [153]:
rew['rew_pre'].iloc[find_pos_idx(('득', 'NNG'))].iloc[0]

'우선 나는 토너성애자다 여름만 되면 토너만 바르는 경향 매우 큰데 안티포어의 경우 나의 니즈를 채워주기 충분했다 최근구매한 제품 중 나의 의식을 일 깨워 준 것이 클XX스의 토너였다 그 제품 또한 매우 마음에 들었지만 이것은 더 큰만족을 주는 제품이었다. 1. 펌핑 타입 이게 아니면 너무 귀찮다 근데 거의 대부분이 펌핑타입이 아니다. 이유는 모르겠지만 말이다 여하튼 펌핑타입이라 빠르게 두세번 촵촵해서 바르기 좋다. 2. 쫀득한 제형 토너라고 다 묽을 쏘냐 노노노 적당히 쫀득거리는 느낌 너무 좋다 줄줄 흘러내리지 않고 흡수가 밀착되어 너무 잘된다 3. 남자꺼라고 저 언혀 요즘 남자의 화장품이라고 아저씨스러운 제품이 많은데 적정선의 아이덴티티를 유지하고 있다 특히 향 시원하면서 고급진 향기라 매우 마음에 든다 4. 토너만 발라도 굿 토너만 발라고 피부. 뒤집어지지 않고 차분해진다 바쁠 때는 얘만 발로도 짱이다 단점도 안쓸순 없겠지 우선 펌핑이 즐거워 넘나 빨리 쓸 것 같은 기분 남자 전용이라는 이미지가 좀 부족해서 나는 남자니까 요걸 써야지라는 자발적인 의식이 생겨나긴 힘들듯 화장품 남자코너에 뙇 있지 않는 한 요즘 좋은 남자용 화장품이 많이 나와서 행복하다'

In [158]:
k.pos('노노노 적당히 쫀득 거리는 느낌 너무 좋다 줄줄')

[('노', 'NNG'),
 ('노', 'NNG'),
 ('노', 'NNG'),
 ('적당히', 'MAG'),
 ('쫄', 'VV'),
 ('ㄴ', 'ETD'),
 ('득', 'NNG'),
 ('거리', 'NNG'),
 ('는', 'JX'),
 ('느낌', 'NNG'),
 ('너무', 'MAG'),
 ('좋', 'VA'),
 ('다', 'ECS'),
 ('줄줄', 'MAG')]

In [177]:
c.loc[c['word']=='똑같']

,word,pos,count
1258,똑같,VA,29
